In [8]:
import pandas as pd
import re

# Load the Excel file
file_path = 'AYG_Events.xlsx'

# Read the sheet
df = pd.read_excel(file_path, sheet_name='eventsTemplateMap')

print(f"Loaded {len(df)} rows from eventsTemplateMap")

def extract_gender_and_format_event(event_name):
    """
    Extract gender from event name and format event to match competition schedule
    Returns: (gender, formatted_event)
    """
    if pd.isna(event_name) or event_name == '':
        return ('', '')
    
    event_str = str(event_name)
    gender = ''
    event = event_str
    
    # Extract gender prefix
    if event_str.startswith('Men '):
        gender = 'MEN'
        event = event_str[4:]  # Remove "Men "
    elif event_str.startswith('Women '):
        gender = 'WOMEN'
        event = event_str[6:]  # Remove "Women "
    elif event_str.startswith('Mixed '):
        gender = 'MIXED'
        event = event_str[6:]  # Remove "Mixed "
    
    # Convert to uppercase
    event = event.upper()
    
    # Standardize relay formats
    event = re.sub(r'4\s*[Xx]\s*100[Mm]', '4X100M', event)
    
    # Standardize distance measurements
    event = re.sub(r'(\d+)\s*[Mm]\b', r'\1M', event)
    
    # Standardize weight measurements
    event = re.sub(r'(\d+)\s*-\s*(\d+)\s*[Kk][Gg]', r'\1 - \2KG', event)
    event = re.sub(r'UNDER\s+(\d+)\s*[Kk][Gg]', r'UNDER \1KG', event)
    event = re.sub(r'OVER\s+(\d+)\s*-\s*(\d+)\s*[Kk][Gg]', r'OVER \1 - \2KG', event)
    
    # Standardize hurdles heights
    event = re.sub(r'\((\d+\.?\d*)\s*[Cc][Mm]\)', r'(\1CM)', event)
    
    # Standardize throw weights
    event = re.sub(r'\((\d+\.?\d*)\s*[Kk][Gg]\)', r'(\1KG)', event)
    event = re.sub(r'\((\d+)\s*[Gg]\)', r'(\1G)', event)
    
    # Clean up multiple spaces
    event = re.sub(r'\s+', ' ', event).strip()
    
    return (gender, event)

def standardize_rounds(rounds_str):
    """
    Standardize rounds format to match competition schedule
    """
    if pd.isna(rounds_str) or rounds_str == '':
        return ''
    
    rounds_mapping = {
        'Heats': 'HEATS',
        'Finals': 'FINALS',
        'Final': 'FINALS',
        'Semi-Finals': 'SEMIFINALS',
        'Semifinals': 'SEMIFINALS',
        'Quarter Finals': 'QUARTERFINALS',
        'Quarterfinals': 'QUARTERFINALS',
        'Round 1': 'ROUND 1',
        'Round 2': 'ROUND 2',
        'Round 64': 'ROUND OF 64',
        'Round 32': 'ROUND OF 32',
        'Round 16': 'ROUND OF 16',
        'Round of 64': 'ROUND OF 64',
        'Round of 32': 'ROUND OF 32',
        'Round of 16': 'ROUND OF 16',
        'Round of 8': 'ROUND OF 8',
        'Preliminary': 'PRELIMINARY',
        'Preliminary Stage Round 1': 'PRELIMINARY STAGE ROUND 1',
        'Preliminary Stage Round 2': 'PRELIMINARY STAGE ROUND 2',
        'Preliminary Stage Round 3': 'PRELIMINARY STAGE ROUND 3',
        'Qualifications': 'QUALIFICATIONS',
        'Repechages': 'REPECHAGES',
        '3rd Places': 'BRONZE MEDAL MATCH',
        'Direct Eliminations': 'DIRECT ELIMINATION',
        'Group Stage': 'GROUP STAGE',
        'Final Round': 'FINAL ROUND',
        '1st Round': 'ROUND 1',
        '2nd Round': 'ROUND 2',
    }
    
    rounds_str = str(rounds_str).strip()
    
    # Try exact match
    if rounds_str in rounds_mapping:
        return rounds_mapping[rounds_str]
    
    # Try case-insensitive match
    for key, value in rounds_mapping.items():
        if key.upper() == rounds_str.upper():
            return value
    
    # Return as uppercase if no match found
    return rounds_str.upper()

# Process each row
print("Processing rows...")
updated_count = 0

for idx, row in df.iterrows():
    # Extract gender and format event
    gender, formatted_event = extract_gender_and_format_event(row['Event'])
    
    # Update the DataFrame
    df.at[idx, 'Gender'] = gender
    df.at[idx, 'Event'] = formatted_event
    
    # Standardize rounds
    if 'Rounds' in df.columns:
        df.at[idx, 'Rounds'] = standardize_rounds(row['Rounds'])
    
    updated_count += 1

print(f"✅ Processed {updated_count} rows")

# Display sample of updated data
print("\n📊 Sample of updated data:")
print(df[['Sport', 'Discipline', 'Event', 'Rounds', 'Gender']].head(15))

# Save back to Excel
try:
    # Load all sheets
    with pd.ExcelFile(file_path) as xls:
        sheets_dict = {sheet_name: pd.read_excel(xls, sheet_name) for sheet_name in xls.sheet_names}
    
    # Update the specific sheet
    sheets_dict['uniqueEventsExploded_v1'] = df
    
    # Write all sheets back
    with pd.ExcelWriter(file_path, engine='openpyxl', mode='w') as writer:
        for sheet_name, sheet_df in sheets_dict.items():
            sheet_df.to_excel(writer, sheet_name=sheet_name, index=False)
    
    print(f"\n✅ Successfully updated {file_path}")
    
except Exception as e:
    print(f"\n❌ Error saving to original file: {str(e)}")
    print("Saving to new file...")
    
    # Save to new file
    output_file = 'AYG_Events_updated.xlsx'
    df.to_excel(output_file, sheet_name='eventsTemplateMap', index=False)
    print(f"✅ Data saved to: {output_file}")

# Verification
print("\n" + "="*50)
print("VERIFICATION REPORT")
print("="*50)

# Check for empty gender values
empty_gender = df[df['Gender'].isna() | (df['Gender'] == '')]
if len(empty_gender) > 0:
    print(f"\n⚠️  Warning: {len(empty_gender)} rows with empty gender:")
    print(empty_gender[['Sport', 'Event', 'Rounds']].to_string())
else:
    print(f"\n✅ All {len(df)} rows have gender values")

# Display statistics
print("\n📈 Gender Distribution:")
print(df['Gender'].value_counts())

print("\n📈 Sample Events by Gender:")
for gender in ['MEN', 'WOMEN', 'MIXED']:
    gender_df = df[df['Gender'] == gender]
    if len(gender_df) > 0:
        print(f"\n{gender}:")
        print(gender_df[['Event', 'Rounds']].head(3).to_string(index=False))

print("\n✅ Processing complete!")


Loaded 163 rows from eventsTemplateMap
Processing rows...
✅ Processed 163 rows

📊 Sample of updated data:
       Sport Discipline              Event  Rounds Gender
0   Aquatics   Swimming    100M BACKSTROKE   HEATS    MEN
1   Aquatics   Swimming    100M BACKSTROKE  FINALS    MEN
2   Aquatics   Swimming  100M BREASTSTROKE   HEATS    MEN
3   Aquatics   Swimming  100M BREASTSTROKE  FINALS    MEN
4   Aquatics   Swimming     100M BUTTERFLY   HEATS    MEN
5   Aquatics   Swimming     100M BUTTERFLY  FINALS    MEN
6   Aquatics   Swimming     100M FREESTYLE   HEATS    MEN
7   Aquatics   Swimming     100M FREESTYLE  FINALS    MEN
8   Aquatics   Swimming    200M BACKSTROKE   HEATS    MEN
9   Aquatics   Swimming    200M BACKSTROKE  FINALS    MEN
10  Aquatics   Swimming  200M BREASTSTROKE   HEATS    MEN
11  Aquatics   Swimming  200M BREASTSTROKE  FINALS    MEN
12  Aquatics   Swimming     200M BUTTERFLY   HEATS    MEN
13  Aquatics   Swimming     200M BUTTERFLY  FINALS    MEN
14  Aquatics   Swimming 